In [ ]:
%load_ext autoreload
%autoreload 2

https://wiki.fysik.dtu.dk/ase/ase/calculators/espresso.html

https://github.com/Yavar-Azar/Courses/tree/main/QE/ASE-QE

In [ ]:
from ase.io import read, write
from ase.calculators.espresso import Espresso

In [ ]:
crystal=read("test/MIN-167-350K-CuCbPyz.cif")

In [ ]:
from ase.visualize import view

view(crystal)

In [ ]:
# inp_data={'prefix':"myprefix",'electron_maxstep':200,'outdir':"./",        'pseudo_dir':"./",'tstress':True,'tprnfor':True,'calculation':'scf', 
#  'ecutrho':240,'verbosity':'high','ecutwfc':30, 'diagonalization': 'david', 'occupations':'smearing','smearing':'mp', 'mixing_mode':'plain', 'mixing_beta':0.7,'degauss':0.001, 'nspin':1}

# pseudodict={"Na":"", "As":"As.pbe-n-rrkjus_psl.1.0.0.UPF"}

In [ ]:
import os
os.chdir('../..')
path = os.getcwd()
print(path)


In [ ]:
head_dir = os.path.abspath("")
print(head_dir)

In [ ]:
# !export ASE_ESPRESSO_COMMAND="mpirun -np 10 pw.x -in PREFIX.pwi > PREFIX.pwo"; 
# !echo $ASE_ESPRESSO_COMMAND

In [ ]:
!pkill -9 -f pw.x

In [ ]:
from ase import units

In [ ]:
%%time

from ase.io import read, write

from ase.build import bulk
from ase.calculators.espresso import Espresso
from ase.constraints import UnitCellFilter
from ase.optimize import LBFGS

import os
from datetime import datetime
# os.makedirs('test', exist_ok=True)
# os.chdir('test')

os.chdir(head_dir)
# crystal=read("test/MIN-167-350K-CuCbPyz.cif")
tms = read("test/tms.xyz")

crystal = tms
a = 20.0
crystal.set_cell([a, a, a])
crystal.set_cell([(a, 0, 0), (0, a, 0), (0, 0, a)])

dir_name = f'{datetime.now().strftime("%Y-%m-%d_%I %M-%S_%p")}'
dir_name = 'test/' + dir_name
os.makedirs(dir_name, exist_ok=False)
os.chdir(dir_name)


pseudo_dir = '../../Pseudopotentials'



# crystal = bulk('NaCl', crystalstructure='rocksalt', a=6.0)


# calc = Espresso(pseudopotentials=pseudopotentials, pseudo_dir = pseudo_dir,
#                 tstress=True, tprnfor=True, ecutwfc=10, kpts=(3, 3, 3))
# crystal.calc=Espresso(pseudopotentials=pseudodict,input_data=inp_data, kpts=(1,1,1))

calc = Espresso(pseudopotentials=pseudopotentials, pseudo_dir = pseudo_dir,
outdir = './outdir',  prefix = 'crystal', restart_mode = 'from_scratch',
                tstress=True, tprnfor=True, nosym=True, 
                ecutwfc=50, kpts=(1, 1, 1), 
                ecutrho = 500,
                occupations = 'smearing', smearing = 'gauss', degauss = 1.0e-2
                )

# write("input.pwi", crystal, pseudopotentials=pseudodict, input_data=inp_data, kpts=(1,1,1))

crystal.calc = calc

# ucf = UnitCellFilter(rocksalt)
# opt = LBFGS(ucf)
# opt.run(fmax=0.005)

# # cubic lattic constant
# print((8*rocksalt.get_volume()/len(rocksalt))**(1.0/3.0))

print(crystal.get_potential_energy())
# print(rocksalt.get_forces())


In [ ]:
from runner import pw_runner

from ase.io import read, write

In [ ]:
tms = read("test/tms.xyz")
a = 20.0
crystal = tms
crystal.set_cell([a, a, a])
crystal.set_cell([(a, 0, 0), (0, a, 0), (0, 0, a)])

calc_dir = pw_runner.make_calc_dir()
pw_runner.run_pw_scf(calc_dir, crystal, num_proc_pw=1)

print(calc_dir)

In [ ]:
pw_runner.run_gipaw(calc_dir, num_proc_gipaw=1)

In [ ]:
read("espresso_gipaw.pwo", format="espresso-out")

In [ ]:
lines = """
     f-sum rule (1st term):
           -31.4288         -0.0003         -0.0011
            -0.0004        -31.4324          0.0004
            -0.0013          0.0004        -31.4335

     f-sum rule (2nd term):
             0.0000          0.0000          0.0000
             0.0000          0.0000          0.0000
             0.0000          0.0000          0.0000

     f-sum rule (should be   -32.0000):
           -31.4288         -0.0003         -0.0011
            -0.0004        -31.4324          0.0004
            -0.0013          0.0004        -31.4335

     chi_bare pGv (HH) in paratec units:
        12.267088      0.019349      0.011588
         0.019529     12.185418     -0.001023
         0.011661     -0.002332     12.193622

        12.267088      0.019349      0.011588
         0.019529     12.185418     -0.001023
         0.011661     -0.002332     12.193622

     chi_bare vGv (VV) in paratec units:
         8.268179      0.020617      0.013567
         0.018959      8.166951     -0.003551
         0.012138     -0.004334      8.174174

         8.268179      0.020617      0.013567
         0.018959      8.166951     -0.003551
         0.012138     -0.004334      8.174174

     Atom  1  Si  pos: (  0.000005 -0.000005  0.000005)  Total sigma:         320.02
           320.0429          0.0166         -0.0018
            -0.1347        320.0497          0.1292
            -0.1303          0.0377        319.9605

     Si   1    anisotropy:      0.22    eta:    0.0000
     Si   1    sigma_11=    319.99    axis=(  0.769763  0.633927  0.074844)
     Si   1    sigma_22=    319.90    axis=(  0.246827 -0.403726  0.880955)
     Si   1    sigma_33=    320.16    axis=( -0.588677  0.659653  0.467244)

     Atom  2  C   pos: ( -0.046155 -0.008570 -0.081165)  Total sigma:         243.30
           242.7726          0.2535          2.4323
             0.2620        241.4559          0.4275
             2.4117          0.4315        245.6826

     C    2    anisotropy:      5.69    eta:   -0.0072
     C    2    sigma_11=    241.42    axis=( -0.463970 -0.815557  0.345830)
     C    2    sigma_22=    241.39    axis=(  0.737523 -0.571883 -0.359178)
     C    2    sigma_33=    247.10    axis=(  0.490705  0.088410  0.866829)

     Atom  3  C   pos: (  0.044415  0.082550  0.002135)  Total sigma:         243.31
           242.6750          2.3709          0.0354
             2.3434        245.8207          0.1084
             0.0490          0.1023        241.4240

     C    3    anisotropy:      5.67    eta:   -0.0069
     C    3    sigma_11=    241.43    axis=( -0.531272  0.265975  0.804368)
     C    3    sigma_22=    241.40    axis=( -0.703848  0.389875 -0.593798)
     C    3    sigma_33=    247.08    axis=( -0.471538 -0.881620 -0.019924)

     Atom  4  C   pos: (  0.062595 -0.069370  0.007840)  Total sigma:         243.30
           243.9723         -2.8097          0.3216
            -2.7997        244.5278         -0.3306
             0.2941         -0.3309        241.4098

     C    4    anisotropy:      5.70    eta:   -0.0155
     C    4    sigma_11=    241.43    axis=( -0.731541 -0.673885 -0.103571)
     C    4    sigma_22=    241.37    axis=( -0.129813 -0.011462  0.991472)
     C    4    sigma_33=    247.10    axis=(  0.669325 -0.738748  0.079094)

     Atom  5  C   pos: ( -0.060860 -0.004610  0.071180)  Total sigma:         243.29
           243.8215          0.1933         -2.8092
             0.1567        241.3663         -0.1934
            -2.7924         -0.1987        244.6859
"""


# TODO: adopt cclig parser: https://github.com/cclib/cclib/blob/master/cclib/parser/orcaparser.py#L1329

In [ ]:
calc_dir

In [ ]:
import psutil
 
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

In [ ]:
os.chdir(head_dir)
crystal=read("test/MIN-167-350K-CuCbPyz.cif")

view(tms)


In [ ]:
print( crystal.get_cell() )
print( crystal.get_pbc() )

crystal=read("test/MIN-167-350K-CuCbPyz.cif")
crystal.get_forces()

In [ ]:
view(crystal)